In [ ]:
import math

import os
import glob as gb
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import load_model
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dropout, Activation,\
               BatchNormalization, Conv2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix                  # pip install mlxtend
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support

from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from operator import truediv


## Import and prepare the dataset

In [ ]:
def image_datagenerator(trainpath, img_height=224, img_width=224, batch_size=20):

    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # set validation split
    test_datagen = ImageDataGenerator(rescale=1./255)

    print("The data is being split into training and validation set")

    train_generator = train_datagen.flow_from_directory(
    trainpath,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training') # set as training data
    print("----------------------------------------------------------------")

    validation_generator = train_datagen.flow_from_directory(
    trainpath, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation') # set as validation data

    # check the number of images in each class in the training dataset
    No_images_per_class = []
    training_class = []

    for i in os.listdir (trainpath):         
        Class_name = os.listdir(os.path.join(trainpath, i))
        No_images_per_class.append(len(Class_name))
        training_class.append(i)
        print('Number of images in {} = {} \n'.format(i, len(Class_name)))


    return train_generator, validation_generator, training_class

### Import the test set

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
img_height=224
img_width=224

testpath= ('................./test_set/')

test_generator = test_datagen.flow_from_directory(
        testpath,# This is the target directory
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        shuffle=False,
        seed=42)

# check the number of images in each class in the test dataset
No_images_per_class = []
test_class = []

for i in os.listdir (testpath):         
    Class_name = os.listdir(os.path.join(testpath, i))
    No_images_per_class.append(len(Class_name))
    test_class.append(i)
    print('Number of images in {} = {} \n'.format(i, len(Class_name)))

test_classes = test_generator.class_indices
print('test_classes: ',test_classes)

In [ ]:
class The_proccess():

    def __init__(self):
        self.method = input("Which process do you want to use? \n\n 1) pretrained-imagenet. \n\n 2) CLOG-CD. \n\n 3) Without curriculum learning in pretext model. \n\n 4) CURVETE \n\n Please enter the corresponding number and hit enter >>>>> ")

        if self.method == str(1):
            self.first()
        elif self.method == str(2):
            self.second()
        elif self.method == str(3):
            self.third()
        elif self.method == str(4):
            self.forth()

    def first(self):
        print("Transfer learning with a pretrained-imagenet \n")
        Level=-1
        ImageNet_model(Dataset_path, G, index, Level, Process= 'pretrained-imagenet')

    def second(self):
        print("Training CLOG-CD \n")
        Level=0
        CLOGCD (Dataset_path, G, index, Level, Process='CLOG-CD')

    def third(self):
        print("Training CURVET (WO/CL, W/SD) \n")
        Level=0
        CURVETE_WOCL_WSD (Dataset_path, pseudo_labels_path, G, index, Level, Process='CURVET (WOCL_WSD)')

    def forth(self):
        print("Training CURVET model\n")
        Level=0
        CURVETE (Dataset_path, pseudo_labels_path, G, index, Level, Process='CURVETE_model')




### Traditional Transfer Learning

In [ ]:
def ImageNet_model(Dataset_path, granularity, index, Level, Process):
    """
    Trains a DenseNet121-based model on the dataset at the given granularity level.

    Args:
        Dataset_path (str): Path to the dataset (labelled or pseudo-labelled).
        granularity (list): List of granularity levels, e.g. ['g_5','g_4','g_3','g_2','g_1'].
        index (list): List of decomposition indices, matching the granularity list, e.g. [5,4,3,2,1].
        Level (int): Current granularity level index (0 = g_5).
        Process (str): Label for saving/tracking the training process.
    """

    # Select granularity folder (e.g., g_5, g_4, etc.)
    folder = granularity[Level]
    index_value = index[Level]   # index associated with this granularity level
    granularity_path = os.path.join(Dataset_path, folder)

    print(f"\n Training at granularity {folder} (index={index_value}) for process {Process}\n")

    # Prepare data generators for training and validation
    train_generator, validation_generator, training_class = image_datagenerator(granularity_path)

    # Build the DenseNet121-based model
    base_model = DenseNet121(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(1024, activation='relu')(x)
    predictions = layers.Dense(len(training_class), activation='softmax')(x)
    base_model = Model(inputs=base_model.input, outputs=predictions)

    # Train the model using the given data
    finetuned_model, learned_weights, save_here = Training_model(
        base_model, train_generator, validation_generator, training_class,
        folder, index_value, Process
    )

    return finetuned_model, learned_weights


### Transfer previous knowledge

In [ ]:
def TransferLearning(TransfereLearned_Model, learned_weights, train_generator, validation_generator,
                      training_class, next_level, index, Process):
    """
    This function modifies a pre-trained model to adapt it for a new classification task.
    It loads the learned weights from the previous level, replaces the output layer with a new dense layer for the
    current training class, and trains the model using provided data generators.
    Returns:
    The fine-tuned model, the learned weights, and the path to save the model.
    """

    # Load the learned weights into the pre-trained model
    TransfereLearned_Model.load_weights(learned_weights)  # Load weights from the specified path

    # Create a new model that outputs from the second-to-last layer of the pre-trained model
    TransfereLearned_Model = Model(TransfereLearned_Model.input, TransfereLearned_Model.layers[-2].output)

    # Add a new classification output layer for the new task
    new_prediction = layers.Dense(len(training_class), activation='softmax', name="new_task")(TransfereLearned_Model.output)

    TransfereLearned_Model = Model(inputs=TransfereLearned_Model.input, outputs=new_prediction)

    # Train the model using the provided training and validation generators
    finetuned_model, learned_weights, save_here = Training_model(
        TransfereLearned_Model, train_generator, validation_generator,
        training_class, next_level, index, Process
    )

    return finetuned_model, learned_weights, save_here


In [ ]:
def CurriculumDecomp(model, weights, Dataset_path, granularity, index, Level, Process, skip_first=False):
    """
    Curriculum learning with class decomposition across different granularity levels.
    - If skip_first=True: used by CLOG-CD.
    - If skip_first=False: used by CURVETE_WOCL_WSD downstream.
    """
    model.load_weights(weights)
    num_iter = 0
    delta = 1

    curr = Level
    max_iterations = 20
    
    while num_iter < max_iterations:
        num_iter += 1

        # Forward curriculum
        forward_indices = list(range(curr, len(granularity)))
        if skip_first and forward_indices:
            forward_indices = forward_indices[1:]  # drop the current level (e.g., skip g_5 for CLOG-CD)

        for idx in forward_indices:
            path_iter = os.path.join(save_to_dir, Process, f"{num_iter}")
            os.makedirs(path_iter, exist_ok=True)

            next_level = granularity[idx]
            next_index = index[idx]
            print(f'Forward → , granularity={next_level}, index={next_index}')

            granularity_path = os.path.join(Dataset_path, next_level)
            train_generator, validation_generator, training_class = image_datagenerator(granularity_path)

            model, weights, _ = TransferLearning(
                model, weights, train_generator, validation_generator,
                training_class, next_level, next_index, Process=path_iter
            )

        # Update current pointer to the last index we used (if any)
        if forward_indices:
            curr = forward_indices[-1]
        else:
            pass

        # Backward curriculum
        num_iter += 1
        backward_indices = list(range(curr - 1, -1, -1))  # go from previous level down to 0

        for idx in backward_indices:
            path_iter = os.path.join(save_to_dir, Process, f"{num_iter}")
            os.makedirs(path_iter, exist_ok=True)

            backward_level = granularity[idx]
            backward_index = index[idx]
            print(f'Backward ← , granularity={backward_level}, index={backward_index}')

            granularity_path = os.path.join(Dataset_path, backward_level)
            train_generator, validation_generator, training_class = image_datagenerator(granularity_path)

            model, weights, _ = TransferLearning(
                model, weights, train_generator, validation_generator,
                training_class, backward_level, backward_index, Process=path_iter
            )

        # Reset for next iteration 
        curr = 0

    return model, weights


### CLOG-CD

In [ ]:
def CLOGCD(Dataset_path, granularity, index, Level, Process):
    """
    Supervised training pipeline: uses ImageNet pre-trained weights.
    Applies curriculum learning and class decomposition on real labeled data.
    """
    model, weights = ImageNet_model(Dataset_path, granularity, index, Level, Process)  # trains starting from g_5
    # Now continue curriculum starting at g_4
    model, weights = CurriculumDecomp(model, weights, Dataset_path,
                                      granularity, index, Level, Process,
                                      skip_first=True)   # means skip g_5 and starts training from g_4
    return model, weights


### CURVET (WO/CL, W/SD)

In [ ]:
def Pretext_SampleDecomp(pseudo_labels_path, granularity, index, Level, Process):
    """
    Pretext training using self-supervised sample decomposition ONLY.
    No curriculum learning applied in this stage.
    Always starts from g_5 (Level=0).
    """

    #Level = 0   # g_5

    model, weights = ImageNet_model(
        Dataset_path=pseudo_labels_path,
        granularity=granularity,
        index=index,
        Level=Level,
        Process=Process + "_pretext_noCL"
    )
    return model, weights


In [ ]:
def CURVETE_WOCL_WSD(Dataset_path, pseudo_labels_path, granularity, index, Level, Process):
    """
    CURVETE (WO/CL, W/SD):

    Stage 1: Pretext task using pseudo-labeled data with sample decomposition only.
             Curriculum learning is NOT used in this stage.

    Stage 2: Downstream fine-tuning using class decomposition + curriculum learning
             with very limited real labeled data.
    """
    
    # Stage 1: Self-supervised sample decomposition (no curriculum)
    print("\n Starting Stage 1: Pretext training (Sample Decomposition only, No Curriculum) \n")
    model, weights = Pretext_SampleDecomp(pseudo_labels_path, granularity, index, Level, Process)

    print("Starting Stage 2: Downstream training with Curriculum Learning + Class Decomposition \n")
    # Start at g_5, index=5 (Level should point to g_5, typically Level=0)
    model, weights = CurriculumDecomp(model, weights, Dataset_path,
                                      granularity, index, Level, Process + "_downstream",
                                      skip_first=False)
    return model, weights


### CURVETE

In [ ]:
def TrainingPseudoLabels(pseudo_labels_path, granularity, index, Level, Process):
    """
    CURVETE Stage 1: Pretext training on pseudo-labeled data (self-supervised).
    Now uses curriculum learning on pseudo-labelled data (g_5 → g_1).
    """
    # Stage 1 starts at g_5 using pseudo-labels
    model, weights = ImageNet_model(
        Dataset_path=pseudo_labels_path,
        granularity=granularity,   
        index=index,
        Level=Level,               
        Process=Process + "_pretext"
    )

    # Continue curriculum: g_4 → g_1
    model, weights = CurriculumDecomp(
        model, weights, pseudo_labels_path,
        granularity, index, Level, Process + "_pretext",
        skip_first=True   # Skip g_5 since it was already trained above
    )

    return model, weights


In [ ]:
def CURVETE(Dataset_path, pseudo_labels_path, granularity, index, Level, Process):
    """
    CURVETE: Self-supervised pretext training (Stage 1) followed by supervised fine-tuning (Stage 2).

    Stage 1: Uses pseudo-labeled data to pretrain the model based curriculum learning and sample decomposition.
    Stage 2: Fine-tunes the pretrained model on a small set of labelled data. Here, the model also trained based
    on curriculum learning combined with different granularity of decomposition.
    """

    # Stage 1 – Self-supervised pretext learning
    print("\n Starting Stage 1: Pretext training on pseudo-labels")
    model, weights = TrainingPseudoLabels(pseudo_labels_path, granularity, index, Level, Process + "_pretext")

    # Stage 2 – Downstream task with real labels
    print("\n Starting Stage 2: Fine-tuning on labeled downstream task")
    # Start at g_5, index=5 (Level should point to g_5, typically Level=0)
    model, weights = CurriculumDecomp(model, weights, Dataset_path,
                                      granularity, index, Level, Process + "_downstream",
                                      skip_first=False)

    return model, weights


### Training the model

In [ ]:
def Training_model(finetuning_model, train_generator, validation_generator, training_class, folder, index, Process):
    """
    This function handles the training process for a fine-tuned model.

    Parameters:
    folder (str): The current folder level for saving weights.
    index (int): The index corresponding to the current training iteration.
    Process (str): A string indicating the type of training process.

    Returns:
    The trained model and the directory for saving outputs.
    """

    # Define the directory to save the weights based on the process type
    save_here = os.path.join(save_to_dir, Process)

    # Create the directory if it does not exist
    if not os.path.exists(save_here):
        os.makedirs(save_here)

#    learned_weights = os.path.join(save_here, 'weights_' + folder + '.h5')
    learned_weights = os.path.join(save_here, f'weights_{folder}.h5')


    # Define checkpoint for saving the best model weights
    checkpoint = ModelCheckpoint(filepath=learned_weights,
                                 monitor='val_accuracy', 
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='max', verbose=1)

    # Early stopping callback to prevent overfitting
    earlystop = EarlyStopping(monitor="val_accuracy", patience=8, mode="auto")

    # Learning rate scheduler to adjust learning rate during training
    def lr_scheduler(epoch):
        initial_lrate = 0.001
        drop = 0.85
        epochs_drop = 15.0
        lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
        return lrate

    lrscheduler = LearningRateScheduler(lr_scheduler)
    callbacks = [checkpoint, earlystop, lrscheduler]

    batch_size = 50

    # Display the number of training and validation samples
    print(f"Number of training samples: {train_generator.samples}")
    print(f"Number of validation samples: {validation_generator.samples}")

    # Calculate steps per epoch based on the size of training data and batch size
    steps_per_epoch = max(1, train_generator.samples // batch_size)
    validation_steps = max(1, validation_generator.samples // batch_size)

    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Validation steps: {validation_steps}")

    # Compile the model with an optimizer, loss function, and metrics
    finetuning_model.compile(optimizer=SGD(), loss="categorical_crossentropy", metrics=["accuracy"])

    # Train the model using the training and validation data generators
    history = finetuning_model.fit(train_generator,
                                   steps_per_epoch=steps_per_epoch,
                                   validation_data=validation_generator,
                                   validation_steps=validation_steps,
                                   epochs=50,
                                   callbacks=callbacks,
                                   verbose=1,
                                   shuffle=True)

    # Get predictions and generate confusion matrix
#    y_true, y_predict = model_prediction(finetuning_model, learned_weights, save_here)
    y_true, y_predict = model_prediction(finetuning_model, learned_weights)

    ConfusionMatrix(y_true, y_predict, save_here, folder, index)

    return finetuning_model, learned_weights, save_here


###  Make a prediction on a test data

In [ ]:
def model_prediction( model, best_learnetweights, batch_size=1):

    x_test , y_test = [] , []
    for i in range(test_generator.n//1):
        a , b = test_generator.next()
        x_test.extend(a)
        y_test.extend(b)
    y_test= np.array(y_test)

    # Predict the output
    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    test_generator.reset()


    # loading the convergence weights
    model.load_weights(best_learnetweights)

    #make prediction
    print('Make a prediction on a test set:')
    y_test_pred= model.predict(test_generator,steps=STEP_SIZE_TEST,verbose=1)
    y_prediction = np.argmax(y_test_pred, axis=1)## predicted_class_indices

    # ground truth labels
    y_true = np.argmax(y_test, axis=1)

    return y_true, y_prediction

### Evaluate the model performance

In [ ]:
"""
- test_classes: List of class names used in the classification task.
- y_predict: Array of predicted labels.
- y_true: Array of true labels.
- folder_name: Identifier for the current experiment or granularity level.
- index: Correction factor applied to predictions when needed.
- save_to_dir: Directory where evaluation results will be saved.
"""

def ConfusionMatrix (y_true, y_predict, save_here, folder_name, index):       #get confusion matrix

    if folder_name !='k_1':        ###  Refine the final classification using error-correction criteria.
        correct_prediction=[]
        for i in y_predict:
            correct_prediction.append(i // index)
        y_predict=np.array(correct_prediction)


    # get confusion matrix
    cm = confusion_matrix(y_true, y_predict)
    class_names = [cls for cls, idx in sorted(test_classes.items(), key=lambda x: x[1])]

    # plot
    fig, ax = plot_confusion_matrix(conf_mat=cm,  figsize=(6, 6),
                            colorbar=False,
                            show_absolute=True,
                            show_normed=False,
                            class_names=test_classes,cmap="Blues")
    # save the figure   
    cm_filename = f'ConfusionMatrix_{folder_name}_iter{index}.png'
    plt.savefig(os.path.join(save_here, cm_filename))
    plt.show()
    
    
    # Classification report
    report = classification_report(y_true, y_predict, target_names=class_names, digits=4, output_dict=True)
    accuracy = accuracy_score(y_true, y_predict)

    # Print summary
    print(classification_report(y_true, y_predict, target_names=class_names, digits=4))
    print('Overall accuracy= ', accuracy)

    # Save confusion matrix values as CSV
    #cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    #cm_csv = f'ConfusionMatrix_{folder_name}_iter{index}.csv'
    #cm_df.to_csv(os.path.join(save_here, cm_csv))

    # Save classification report as CSV
    report_df = pd.DataFrame(report).transpose()
    report_csv = f'ClassificationReport_{folder_name}_iter{index}.csv'
    report_df.to_csv(os.path.join(save_here, report_csv))

   # print(f"Confusion matrix saved to {cm_csv}")
    #print(f"Classification report saved to {report_csv}")

### Main

In [ ]:
# Paths to the dataset and the directory where results will be saved

Dataset_path = '................./labelled_dataset/'
save_to_dir = '................./results/'
pseudo_labels_path= '........../pseudo_labells/'

# Determine the decomposition granularity by listing all the folders in the dataset path
# Each folder represents a granularity level (e.g., g1, g2, ..., g9_max)
decomposition_granularity = os.listdir(Dataset_path)


# Sort the granularity levels in descending order (from g_max to g_min)
# Example: [g5, g4, ..., g1]
decomposition_granularity.sort(reverse=True)

# Initialize lists to store the granularity levels (G) and their corresponding indices

G=[]
index=[]
decomposition_granularity.sort(reverse=True)

for i, folder in list(enumerate(decomposition_granularity,1)):         # G=[9_max,...,g2,g1]
    #print('Index= ',i, '  The granularity decomposition class is: ',folder)
    G.append(folder)     # if max=5 then: G=[9_5, g_4, g_3, g_2, g_1]
    index.append(i)      # if max=5 then:  index=[5, 4, 3, 2, 1]

index.sort(reverse=True)
# Display the sorted granularity levels and corresponding indices
print ('______________________________________________\n')
print('G = \n', decomposition_granularity )
print('index= ', index)


In [ ]:
Select_model = The_proccess()
